In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI 管道：KFP 的管道介绍

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/pipelines/pipelines_intro_kfp.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> 在 Colab 中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Fpipelines%2Fpipelines_intro_kfp.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> 在 Colab Enterprise 中打开
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/pipelines/pipelines_intro_kfp.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> 在 Workbench 中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/pipelines/pipelines_intro_kfp.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> 在 GitHub 上查看
    </a>
  </td>
</table>

## 概述

本笔记本提供了如何使用[Vertex AI Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines)和[Kubeflow Pipelines (KFP) SDK](https://www.kubeflow.org/docs/components/pipelines/)的介绍。

了解更多关于[Vertex AI Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines/introduction)。

### 目标

在本教程中，您将学习如何使用Python的KFP SDK来构建生成评估指标的管道。

本教程使用以下Vertex AI服务：

- Vertex AI Pipelines

执行的步骤包括：

- 定义和编译一个Vertex AI管道。
- 指定要用于管道运行的服务帐号。
- 使用Vertex AI SDK for Python和REST API运行管道。

### 成本

本教程使用 Google Cloud 的计费组件：

* Vertex AI
* 云存储

了解 [Vertex AI 定价](https://cloud.google.com/vertex-ai/pricing)，
[云存储定价](https://cloud.google.com/storage/pricing)，
并使用 [定价计算器](https://cloud.google.com/products/calculator/) 
根据您的预期使用情况生成成本估算。

开始吧

安装Vertex AI SDK for Python和其他必需的包

In [ ]:
# Install the packages
! pip3 install --upgrade google-cloud-aiplatform \
                         google-cloud-storage \
                         kfp \
                         google-cloud-pipeline-components

### 重新启动运行时（仅适用于Colab）

为了使用新安装的包，您必须重新启动Google Colab上的运行时。

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️内核即将重新启动。在继续下一步之前，请等待其完成。⚠️</b>
</div>

检查您安装的软件包版本。KFP SDK 版本应该是 >=1.6。

In [ ]:
! python3 -c "import kfp; print('KFP SDK version: {}'.format(kfp.__version__))"

### 验证你的笔记本环境（仅限Colab）

在Google Colab上验证你的环境。

In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### 设置谷歌云项目信息

了解更多关于[设置项目和开发环境](https://cloud.google.com/vertex-ai/docs/start/cloud-environment)。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

### 创建一个云存储桶

创建一个存储桶来存储诸如数据集等中间产品。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

如果您的存储桶还不存在：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

设置 Vertex AI 管道的服务账户访问权限

运行以下命令，授予您的服务账户在您之前创建的存储桶中读取和写入管道工件的权限。您只需要为每个服务账户运行此步骤一次。

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [ ]:
import sys

IS_COLAB = "google.colab" in sys.modules
if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    else:  # IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

将[*Storage Object Creator*](https://cloud.google.com/iam/docs/understanding-roles#storage.objectCreator)和[*Storage Object Viewer*](https://cloud.google.com/iam/docs/understanding-roles#storage.objectViewer)角色授予您的服务帐户。

In [ ]:
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI

! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

### 导入所需的库

In [ ]:
import json
from typing import NamedTuple

from google.cloud import aiplatform
from kfp import compiler, dsl
from kfp.dsl import component

### 初始化Python的Vertex AI SDK

要开始使用Vertex AI，您必须[启用Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

设置以下常量用于 Vertex AI：

- `API_ENDPOINT`：Vertex AI API 服务的终端节点，用于数据集、模型、作业、流水线和端点服务。
- `PIPELINE_ROOT`：Cloud 存储位置，作为流水线的根输出目录。

In [ ]:
# API service endpoint
API_ENDPOINT = f"{LOCATION}-aiplatform.googleapis.com"
# Pipelne root dir
PIPELINE_ROOT = f"{BUCKET_URI}/pipeline_root/intro"

## 定义基于 Python 函数的管道组件

在本教程中，您将定义一个简单的管道，其中包括三个步骤，每个步骤都被定义为一个组件。

### 定义 *hello world* 组件

首先，基于一个非常简单的 Python 函数定义一个组件。它接受一个字符串输入参数并返回该值作为输出。

注意使用 `@component` 装饰器，该装饰器在评估时将函数编译为 KFP 组件。下面的示例指定了一个组件的基础镜像（`python:3.9`），以及一个组件的 YAML 文件 `hw.yaml`。编译后的组件规范将被写入到 YAML 文件中。（默认基础镜像是 `python:3.7`，也可以使用）。

In [ ]:
@component(base_image="python:3.9")
def hello_world(text: str) -> str:
    print(text)
    return text


compiler.Compiler().compile(hello_world, "hw.yaml")

正如你在下面所看到的，编译这个组件会创建一个[任务工厂函数](https://www.kubeflow.org/docs/components/pipelines/sdk/python-function-components/)，名为 `hello_world`，你可以在定义流水线步骤时使用它。

虽然这里没有展示，但如果你想共享这个组件的定义，或者在其他上下文中使用它，你可以从其 yaml 文件中加载它，如下所示：

`hello_world_op = components.load_component_from_file('./hw.yaml')`

如果你的组件 yaml 文件存储在线上，你也可以使用 `load_component_from_url` 方法。（对于 GitHub 链接，请加载 'raw' 文件。）

### 定义 *两个输出* 组件

下面的第一个组件，即 `two_outputs`，演示了安装一个包。在这种情况下，安装了 `google-cloud-storage` 包。或者，您可以指定一个包含所需安装的基本映像。

**注意：** 组件函数实际上并未使用该包。

`two_outputs` 组件返回两个命名输出。

In [ ]:
@component(packages_to_install=["google-cloud-storage"])
def two_outputs(
    text: str,
) -> NamedTuple(
    "Outputs",
    [
        ("output_one", str),  # Return parameters
        ("output_two", str),
    ],
):
    # the import is not actually used for this simple example, but the import
    # is successful, as it was included in the `packages_to_install` list.
    from google.cloud import storage  # noqa: F401

    o1 = f"output one from text: {text}"
    o2 = f"output two from text: {text}"
    print("output one: {}; output_two: {}".format(o1, o2))
    return (o1, o2)

### 定义*consumer*组件

第三个组件 `consumer` 接受三个字符串输入，打印它们并将它们作为输出返回。

In [ ]:
@component
def consumer(text1: str, text2: str, text3: str) -> str:
    print(f"text1: {text1}; text2: {text2}; text3: {text3}")
    return f"text1: {text1}; text2: {text2}; text3: {text3}"

定义一个使用组件的流水线

接下来，定义一个使用上述三个组件的流水线。

通过评估上述组件定义，您创建了任务工厂函数，这些函数用于流水线定义，用于创建流水线步骤。

该流水线接受一个输入参数，并将该参数作为参数传递给前两个流水线步骤（`hw_task` 和 `two_outputs_task`）。

然后，第三个流水线步骤（`consumer_task`）消费第一步和第二步的输出。因为`hello_world`组件定义只返回一个未命名的输出，您将其称为`hw_task.output`。`two_outputs`任务返回两个命名输出，您可以将其访问为`two_outputs_task.outputs["<output_name>"]`。

**注意：** 在`@dsl.pipeline`装饰器中，您将`PIPELINE_ROOT`定义为用作根目录的Cloud Storage路径。您可以选择跳过它，但在创建流水线运行时必须提供它。

In [ ]:
@dsl.pipeline(
    name="intro-pipeline-unique",
    description="A simple intro pipeline",
    pipeline_root=PIPELINE_ROOT,
)
def pipeline(text: str = "hi there"):
    hw_task = hello_world(text=text)
    two_outputs_task = two_outputs(text=text)
    consumer_task = consumer(  # noqa: F841
        text1=hw_task.output,
        text2=two_outputs_task.outputs["output_one"],
        text3=two_outputs_task.outputs["output_two"],
    )

## 编译管道

接下来，将管道编译为一个JSON文件。

**注意：** 您也可以将管道编译为一个YAML文件，但在使用文件进行REST API时一些REST功能可能无法工作。

In [ ]:
compiler.Compiler().compile(pipeline_func=pipeline, package_path="intro_pipeline.json")

运行管道。

In [ ]:
DISPLAY_NAME = "intro_pipeline_job_unique"

job = aiplatform.PipelineJob(
    display_name=DISPLAY_NAME,
    template_path="intro_pipeline.json",
    pipeline_root=PIPELINE_ROOT,
)

job.run()

点击生成的链接在云控制台上查看您的运行情况。

在UI界面中，当您点击节点时，很多流水线DAG节点会展开或折叠。以下是DAG的部分展开视图（点击图像查看更大版本）。

<a href="https://storage.googleapis.com/amy-jo/images/mp/intro_pipeline.png" target="_blank"><img src="https://storage.googleapis.com/amy-jo/images/mp/intro_pipeline.png" width="60%"/></a>

### 删除流水线作业

您可以使用`delete()`方法删除流水线作业。

In [ ]:
job.delete()

## 为管道运行指定服务帐号

默认情况下，用于管道运行的[服务帐号](https://cloud.google.com/iam/docs/service-accounts)是您的[默认计算引擎服务帐号](https://cloud.google.com/compute/docs/access/service-accounts#default_service_account)。
但是，您可能希望以具有访问不同角色的权限来运行管道，而不是为默认服务帐号配置的那些角色。例如，您可能需要使用一组更受限制的权限。

创建并配置服务帐号后，将其作为参数传递给`create_run_from_job_spec`方法。管道作业将以给定服务帐号的权限运行。

了解更多关于[创建和配置服务帐号以与Vertex AI管道配合使用](https://cloud.google.com/vertex-ai/docs/pipelines/configure-project#service-account)。

In [ ]:
DISPLAY_NAME = "intro_pipeline_job_svc_acc"

job = aiplatform.PipelineJob(
    display_name=DISPLAY_NAME,
    template_path="intro_pipeline.json",
    pipeline_root=PIPELINE_ROOT,
)

job.run(
    service_account=SERVICE_ACCOUNT
)  # <-- CHANGE to use non-default service account

删除管道作业

您可以使用`delete（）`方法删除管道作业。

In [ ]:
job.delete()

## 流水线步骤缓存

默认情况下，流水线步骤缓存是启用的。这意味着在可能的情况下重用先前步骤执行的结果。

如果你想禁用流水线运行的缓存，你可以在创建`PipelineJob`对象时将`enable_caching`参数设置为**False**。

尝试两次提交流水线作业：一次将`enable_caching`设置为**True**，另一次将`enable_caching`设置为**False**。

In [ ]:
job = aiplatform.PipelineJob(
    display_name="intro_pipeline_job_cached_unique",
    template_path="intro_pipeline.json",
    enable_caching=False,
)

job.run()

删除管道作业

您可以使用 `delete()` 方法来删除管道作业。

In [ ]:
job.delete()

使用Pipelines REST API

有时候您可能希望使用REST API而不是Python KFP SDK。以下是如何使用REST API的示例。

在某些命令中需要一个流水线ID，您可以从下面显示的流水线列表中的“运行”列中获取这些数据，以及从特定流水线的“详情”页面中获取。您可以使用流水线作业API的列表方法查看流水线详情。

<a href="https://storage.googleapis.com/amy-jo/images/mp/pipeline_id.png" target="_blank"><img src="https://storage.googleapis.com/amy-jo/images/mp/pipeline_id.png" width="80%"/></a>

### 列出管道作业

**注意：** 如果您有很多管道运行，此请求可能会生成大量响应。

In [ ]:
! curl -X GET -H "Authorization: Bearer $(gcloud auth print-access-token)" https://{API_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/pipelineJobs

创建一个管道工作

为了发送一个创建管道工作的REST请求，您需要包括管道工作规范的细节。

因此，请加载先前编译的管道规范细节至一个JSON对象，并将其包括在您的管道配置中。

In [ ]:
# Load the pipeline specification
with open("intro_pipeline.json") as fp:
    pipeline_job_spec = json.load(fp)

In [ ]:
# Specify the pipeline configuration details
pipeline_config = {
    "displayName": "intro-pipeline-rest-api",
    "runtimeConfig": {
        "gcsOutputDirectory": PIPELINE_ROOT,
    },
    "pipelineSpec": pipeline_job_spec,
}

# Save the configuration to a json file
with open("pipeline_config.json", "w") as fp:
    json.dump(pipeline_config, fp)

In [ ]:
# Set a job ID (optional)
PIPELINE_RUN_ID = "intro-pipeline-job-unique"

# Send the job creation request using the configuration payload
output = ! curl -X POST  -H "Authorization: Bearer $(gcloud auth print-access-token)" -H "Content-Type: application/json; charset=utf-8"   https://{API_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/pipelineJobs?pipelineJobId={PIPELINE_RUN_ID}  --data "@pipeline_config.json"

# In case you didn't use a pre-defined PipelineJobId, Vertex AI
# generates one automatically. In such a case, use the following
# commented code to retrieve the generated job id.
# output_json = json.loads(" ".join(output))
# PIPELINE_RUN_ID = output_json['name'].split("/")[-1]
# print(PIPELINE_RUN_ID)

### 使用ID获取管道工作详情

In [ ]:
! curl -X GET -H "Authorization: Bearer $(gcloud auth print-access-token)"  https://{API_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/pipelineJobs/{PIPELINE_RUN_ID}

### 使用ID取消管道作业

**注意：**如果您的管道在您到达这一步之前已成功执行，您会遇到一个错误响应，说明相同的问题。

In [ ]:
! curl -X POST -H "Authorization: Bearer $(gcloud auth print-access-token)" https://{API_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/pipelineJobs/{PIPELINE_RUN_ID}:cancel

### 使用ID删除流水线作业

In [ ]:
! curl -X DELETE -H "Authorization: Bearer $(gcloud auth print-access-token)"  https://{API_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/pipelineJobs/{PIPELINE_RUN_ID}

清理

要清理此项目中使用的所有Google Cloud资源，您可以删除用于教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在此教程中创建的各个资源。

In [ ]:
# Delete the Cloud Storage bucket
delete_bucket = False  # Set True for deletion

if delete_bucket:
    ! gsutil rm -r $BUCKET_URI

# Delete the locally generated files
! rm intro_pipeline.json
! rm pipeline_config.json